In [ ]:
## file handling libraries
import sys
import pandas as pd
import os 
import numpy as np
## importing libraries for
## Vizualization
import matplotlib.pyplot as plt
import seaborn as sb

## loading datasets
dataFrame = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_train.csv')
testFrame = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_test.csv')

## plotting correlations
plt.figure(figsize= (10, 7))
sb.heatmap(dataFrame.corr(), annot = True)

In [ ]:
## for the purpouse our Analysis lets divide the training data into
## 2 parts : those who buy and those who does not buy the policy
## and then see what causing this to happen.

thoseWhoBought = dataFrame[dataFrame['Response'] == 1]
thoseWhoDoesNotBought = dataFrame[dataFrame['Response'] == 0]

plt.pie ([len(thoseWhoBought), len(thoseWhoDoesNotBought)], labels = ['those who bought Policy', 
                                                                      'those who doesnt bought Policy'],autopct='%1.2f%%')

## only 16.38 % people from the survey bought 
## while 83.62 % does not

## This means and model which says nobody bought
## policy is 83.62 accuracte !!!

In [ ]:
## lets comapritively study bought of these categories
## we start this by age distribution
plt.figure(figsize = (14, 5))

plt.subplot(1, 2, 1)
sb.distplot(thoseWhoBought['Age'], kde = False)
plt.title('Age distribution : those who bought')
plt.grid(True)

plt.subplot(1, 2, 2)
sb.distplot(thoseWhoDoesNotBought['Age'], kde = False)
plt.title('Age dist those who doesnt bought')
plt.grid(True)

## This give us a key insigth that
## that people in the early are not
## as likely to buy insurance as those
## in late years (30 - 55)

In [ ]:
## let see what role gender has to play
plt.figure(figsize = (14, 5))

plt.subplot(1, 2, 1)
plt.pie([len (thoseWhoBought[thoseWhoBought['Gender'] == 'Male']),
        len (thoseWhoBought[thoseWhoBought['Gender'] == 'Female'])], labels = ['Males', 
                                                                               'Females'],autopct='%1.2f%%')
plt.title('Gender distribution : those who bought')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.pie([len(thoseWhoDoesNotBought[thoseWhoDoesNotBought['Gender']== 'Male']),
        len(thoseWhoDoesNotBought[thoseWhoDoesNotBought['Gender']== 'Female'])],labels = ['Males', 
                                                                               'Females'],autopct='%1.2f%%' )
plt.title('Gender distribution: those who doesnt bougth')
plt.grid(True)


In [ ]:
## taking about driving lisence 
plt.figure(figsize = (14, 5))

plt.subplot(1, 2, 1)
plt.pie([len (thoseWhoBought[thoseWhoBought['Driving_License'] == 1]),
        len (thoseWhoBought[thoseWhoBought['Driving_License'] == 0 ])], labels = ['have license', 
                                                                               'doesnt have lincese'],autopct='%1.2f%%')
plt.title('Lincese distribution : those who bought')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.pie([len(thoseWhoDoesNotBought[thoseWhoDoesNotBought['Driving_License']== 1]),
        len(thoseWhoDoesNotBought[thoseWhoDoesNotBought['Driving_License']== 0 ])],labels = ['have license', 
                                                                               'doesnt have license'],autopct='%1.2f%%' )
plt.title('Lincese distribution: those who doesnt bougth')
plt.grid(True)


## This means almost every one has a lisence
## Also it isn't easy to get insured without a
## lisence (hence we may safely exclude this feature form our model)

In [ ]:
## let see about regions
plt.figure(figsize = (14, 5))
plt.subplot(1, 2, 1)
sb.distplot(thoseWhoBought['Region_Code'], kde = False)
plt.title('Region_Code distribution : those who bought')
plt.grid(True)

plt.subplot(1, 2, 2)
sb.distplot(thoseWhoDoesNotBought['Region_Code'], kde = False)
plt.title('Region_code: those who doesnt bought')
plt.grid(True)



In [ ]:
## one Last thing before moving to model training part
## let see premium distribution  

plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
sb.distplot(thoseWhoBought['Annual_Premium'], kde = False)
plt.title('Premium distribution : those who bought')
plt.grid(True)

plt.subplot(1, 2, 2)
sb.distplot(thoseWhoDoesNotBought['Annual_Premium'], kde = False)
plt.title('Premium: those who doesnt bought')
plt.grid(True)


In [ ]:
## lets preprocess data

## features we are selecting
featuresNeeded = ['Gender', 'Age', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 
                 'Annual_Premium','Policy_Sales_Channel', 'Vintage' ]

## features that are string must be mapped to int value
featureNeedsToBeMapped = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']

## ids and labels for training
labelsForPrediction = testFrame['id']
trainingResponses = np.array (dataFrame['Response'], dtype = 'float64')


for feature in featureNeedsToBeMapped :
    uvals = set(dataFrame[feature])
    umap = {}
    i = 0
    
    for val in uvals:
        umap[val] = i
        i += 1
    
    dataFrame[feature] = dataFrame[feature].map(umap)
    testFrame[feature] = testFrame[feature].map(umap)

In [ ]:
trainVector = np.array (dataFrame[featuresNeeded], dtype = 'float64')
testVector = np.array (testFrame[featuresNeeded], dtype = 'float64' )

## let us standardize thes values 
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

trainVector = stdsc.fit_transform(trainVector)
testVector = stdsc.transform(testVector)

In [ ]:
## before proceeding to model training lets 
## do dimentionality reduction and plot the 
## scatter of who have bought the policy and
## who does not

from sklearn.decomposition import PCA
pca  = PCA(n_components = 3)
pca.fit(trainVector)

transformedFeatures = pca.transform(trainVector)

In [ ]:
## lets make this a dataFrame
dimReducedDataFrame = pd.DataFrame(transformedFeatures)
dimReducedDataFrame = dimReducedDataFrame.rename(columns = { 0: 'V1', 1 : 'V2', 2 : 'V3'})
dimReducedDataFrame['Response'] = trainingResponses

## Plotting this
plt.figure(figsize = (10, 5))
sb.scatterplot(data = dimReducedDataFrame, x = 'V1', y = 'V2',hue = 'Response')
plt.grid(True)
plt.show()

In [ ]:
## let us split our dataset
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(trainVector, trainingResponses, test_size = 0.1, random_state = 42)

In [ ]:
## lets try model stacking:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import time

estimators = [('gb1', GradientBoostingClassifier(n_estimators= 100) ),
              ('gb2', GradientBoostingClassifier(n_estimators= 50) ),
              ('rf1', RandomForestClassifier(n_estimators= 200) ),
              ('rf2', RandomForestClassifier(n_estimators= 50) ),
              ('dt1', DecisionTreeClassifier()),
              ('bg1', BaggingClassifier())
              
             ]

clf = StackingClassifier(
    estimators=estimators, final_estimator = LogisticRegression())


startTime = time.perf_counter()
clf.fit(xtrain, ytrain)
endTime = time.perf_counter()

print ('Time taken to train : {} seconds'.format(endTime - startTime))
## nearly 26 min needed to train stacked classifier

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print (confusion_matrix(ytest, clf.predict(xtest)))
print (accuracy_score(ytest, clf.predict(xtest)))

In [ ]:
## lets just also try Knn classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(xtrain, ytrain)


confusion_matrix(ytest, knn.predict(xtest))

In [ ]:
## lets also train a neural network
## this model has an accuracy of about 53 %
## when predicting those who doesnt buy the policy
## which is not that great !!! 
import tensorflow as tf
num, featureSize = xtrain.shape

model = tf.keras.models.Sequential([ tf.keras.layers.Dense(45, activation='relu',input_shape = (featureSize, )),
                                 tf.keras.layers.Dense(30, activation='relu'),
                                 tf.keras.layers.Dropout(0.2),tf.keras.layers.Dense(45, activation='relu'),
                                 tf.keras.layers.Dense(1 ,activation='sigmoid')])

model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate= 0.3), loss = 'binary_crossentropy', metrics=['accuracy'])
retObj = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs= 10, batch_size= 100) 

In [ ]:
def AnnPredictions(predVec, model) :
  predictions = model.predict(predVec).flatten()
  predictions[predictions < 0.5] = 0
  predictions[predictions >= 0.5] = 1

  return predictions

In [ ]:
confusion_matrix(ytest, AnnPredictions(xtest, model))
## imbalance prevail !!!

In [ ]:
## but rember we do have pca clustered data which has only 3 dims
## lets check that out

newTrainingVector = np.array(dimReducedDataFrame[['V1', 'V2', 'V3']], dtype = 'float64')
newLabelVector = np.array(dimReducedDataFrame['Response'], dtype = 'float64')
stdc2 = StandardScaler()

newTrainingVector = stdc2.fit_transform(newTrainingVector)

xtrainN, xtestN, ytrainN, ytestN = train_test_split(newTrainingVector, newLabelVector, random_state = 42, test_size = 0.1)


In [ ]:
num, featureSize = xtrainN.shape

model = tf.keras.models.Sequential([ tf.keras.layers.Dense(45, activation='relu',input_shape = (featureSize, )),
                                 tf.keras.layers.Dense(30, activation='relu'),
                                 tf.keras.layers.Dropout(0.2),tf.keras.layers.Dense(45, activation='relu'),
                                 tf.keras.layers.Dense(1 ,activation='sigmoid')])

model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate= 0.3), loss = 'binary_crossentropy', metrics=['accuracy'])
retObj = model.fit(xtrainN, ytrainN, validation_data=(xtestN, ytestN), epochs= 10, batch_size= 100) 

In [ ]:
predictions = clf.predict(testVector)

In [ ]:
## let also do pca clustering and visualize this
pca1  = PCA(n_components = 3)
pca1.fit(testVector)

transformedFeatures1 = pca1.transform(testVector)
dimReducedDataFrame1 = pd.DataFrame(transformedFeatures1)
dimReducedDataFrame1 = dimReducedDataFrame1.rename(columns = { 0: 'V1', 1 : 'V2', 2 : 'V3'})
dimReducedDataFrame1['Response'] = predictions

## Plotting this
plt.figure(figsize = (10, 5))
sb.scatterplot(data = dimReducedDataFrame1, x = 'V1', y = 'V2',hue = 'Response')
plt.grid(True)
plt.show()


In [ ]:
###########################################################################
# work in progress                                                        #
# To try underSampling, and oversampling                                  #
###########################################################################